In [85]:
import requests 
from bs4 import BeautifulSoup
import re # regex expression
import tqdm.notebook as tq # time loop in notebook
import re

In [4]:
def Get_hrefs(url):
    # Request url and gets the hrefs for the movies recommended in the "more like this" section in imdb page for a movie.
    # Returns a list of hrefs elements that link to the movies recommended by imdb
    
    response = requests.get(url)
    result = response.content    
    soup = BeautifulSoup(result, 'html.parser')
    
    # init the list with all href
    hrefs = []
    
    for poster in soup.find_all("div",attrs={'class':'ipc-poster-card ipc-poster-card--base ipc-poster-card--dynamic-width ipc-sub-grid-item ipc-sub-grid-item--span-2'}):
        for link in poster.find_all("a", attrs = {"class":"ipc-lockup-overlay ipc-focusable"}):
            if "href" in link.attrs:
                if link.get("href") not in hrefs:
                    movie_id = link.get("href").split("/?")[0] #only the first part of the link is required as it contains the id of the film. The next sequence might cause errors. 
                    hrefs.append(movie_id) #save the links (ids) in a list.
    return(hrefs)

In [6]:
def get_recommendations(url,depth):
    #Searches in the IMDB "more like this" for recommendations based on the link of one movie. 
    #Input: IMDB link to a movie/show. Depth: amount of iterations to search in the "more like this"
    #Output: a list of links for imdb movies/shows. 
    
    urls_checked = []

    for i in tq.tqdm(range(depth)):
        if i == 0:
            
            hrefs  = Get_hrefs(url)
            urls_checked.append(url) 
        else:
            hrefs_temp = []
            for href in tq.tqdm(hrefs):
                new_url = "https://www.imdb.com" + href
                if new_url not in urls_checked:
                    hrefs_temp += Get_hrefs(new_url)
                    urls_checked.append(new_url)
                    hrefs_temp = list(set(hrefs_temp))
            hrefs += [recom for recom in hrefs_temp if recom not in hrefs]
    
    links = [i.split("title/")[1] for i in hrefs]
    
    return links

In [7]:
x = get_recommendations("https://www.imdb.com/title/tt0372824/",2)
x

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

['tt1064932',
 'tt0120731',
 'tt0119038',
 'tt1754108',
 'tt1063669',
 'tt3547740',
 'tt0249462',
 'tt2800240',
 'tt10186972',
 'tt0364517',
 'tt0039234',
 'tt0301357',
 'tt0314331',
 'tt2147995',
 'tt0948535',
 'tt0929632',
 'tt5657028',
 'tt8162330',
 'tt4085696',
 'tt0114732',
 'tt0119164',
 'tt0100681',
 'tt0120471',
 'tt0039840',
 'tt0765432',
 'tt2066051',
 'tt0044324',
 'tt6859418',
 'tt10366460',
 'tt4400058',
 'tt0426578',
 'tt0096052',
 'tt14485734',
 'tt1369667',
 'tt0118976',
 'tt5736696',
 'tt9770150',
 'tt3421514',
 'tt0083316',
 'tt0106332',
 'tt0250258',
 'tt0032383',
 'tt3478962',
 'tt0344510',
 'tt0213847',
 'tt0038913',
 'tt0105323',
 'tt0494271',
 'tt13349892',
 'tt3013602',
 'tt0078907',
 'tt2987732',
 'tt0109831',
 'tt0039391',
 'tt2044128',
 'tt1529235',
 'tt7476116',
 'tt0384369',
 'tt0405094',
 'tt0114808',
 'tt3530978',
 'tt0248408',
 'tt0012532',
 'tt10405142',
 'tt1440232',
 'tt0286499',
 'tt0039776',
 'tt1793931',
 'tt0474518',
 'tt0039038',
 'tt0408777',
 

In [46]:
#dictionaries must have all numeric information directly in numbers.

movie1 = {"name": "The Matrix",
          "imdb_id": "tt0133093",
         "year_released": 1997,
         "runtime": 136,
          "imdb_reviews": 60,
         "external_reviews": 70,
         "imdb_rating": 8.7,
         "metacritic_punctuation": 73,
          "budget": 63000000,
          "earning_worldwide": 467222728,
          "earning_US&CA": 172076928,
          "genres": ["action","sci-fi"],
          "directors":["Lilly Wachows","kiLana Wachowski"],
          "cast":["keanu reevs", "carrie anne moss","Laurence Fishburne"],
          "imdb_recommendations": {},
          "new_recommendations": {}
         } 

movie2 = {"name": "The Matrix Reloaded",
          "imdb_id": "tt0234215",
         "year_released": 2003,
         "runtime": 138,
          "imdb_reviews": 2900,
         "external_reviews": 184,
         "imdb_rating": 7.2,
         "metacritic_punctuation": 62,
          "budget": 150000000,
          "earning_worldwide": 741847937,
          "earning_US&CA": 281576461,
          "genres": ["action","sci-fi"],
          "directors":["Lilly Wachows","kiLana Wachowski"],
          "cast":["keanu reevs", "carrie anne moss","Helmut Bakaitis"],
          "imdb_recommendations": {},
          "new_recommendations": {}
         } 

In [47]:
def compare_lists(list1,list2):
    #This functions takes two lists and estimates how similar the second list is to the first one. Result might be between 0 and 1.
    list1 = list(set(list1))
    list2 = list(set(list2))
    
    common_elements = len(set(list1) & set(list2))
    
    common_elements_proportion = common_elements / len(list1)
    
    return common_elements_proportion

In [48]:
movie1_vector = [movie1["year_released"],movie1["runtime"],movie1["imdb_reviews"],movie1["external_reviews"],movie1["imdb_rating"],movie1["metacritic_punctuation"],movie1["budget"],
                    movie1["earning_worldwide"],movie1["earning_US&CA"],1,1,1]

In [49]:
movie2_vector = [movie2["year_released"],movie2["runtime"],movie2["imdb_reviews"],movie2["external_reviews"],movie2["imdb_rating"],movie2["metacritic_punctuation"],movie2["budget"],
                    movie2["earning_worldwide"],movie2["earning_US&CA"]]

common_genres_rate = compare_lists(movie1["genres"],movie2["genres"])
movie2_vector.append(common_genres_rate)

common_directors_rate = compare_lists(movie1["directors"],movie2["directors"])
movie2_vector.append(common_directors_rate)

common_cast_rate = compare_lists(movie1["cast"],movie2["cast"])
movie2_vector.append(common_cast_rate)

print(movie1_vector)
print(movie2_vector)

[1997, 136, 60, 70, 8.7, 73, 63000000, 467222728, 172076928, 1, 1, 1]
[2003, 138, 2900, 184, 7.2, 62, 150000000, 741847937, 281576461, 1.0, 1.0, 0.6666666666666666]


In [50]:
from scipy.spatial import distance

In [51]:
euclidean = distance.euclidean(movie1_vector,movie2_vector)
euclidean

308185257.84597576

In [53]:
manhattan = distance.cityblock(movie1_vector,movie2_vector)
type(manhattan)

numpy.float64

In [60]:
def estimateMovieDistance(movie1,movie2):
    movie1_vector = [movie1["year_released"],movie1["runtime"],movie1["imdb_reviews"],movie1["external_reviews"],movie1["imdb_rating"],movie1["metacritic_punctuation"],movie1["budget"],
                    movie1["earning_worldwide"],movie1["earning_US&CA"],1,1,1]
    movie2_vector = [movie2["year_released"],movie2["runtime"],movie2["imdb_reviews"],movie2["external_reviews"],movie2["imdb_rating"],movie2["metacritic_punctuation"],movie2["budget"],
                    movie2["earning_worldwide"],movie2["earning_US&CA"]]
    
    common_genres_rate = compare_lists(movie1["genres"],movie2["genres"])
    movie2_vector.append(common_genres_rate)

    common_directors_rate = compare_lists(movie1["directors"],movie2["directors"])
    movie2_vector.append(common_directors_rate)

    common_cast_rate = compare_lists(movie1["cast"],movie2["cast"])
    movie2_vector.append(common_cast_rate)

    euclidean = distance.euclidean(movie1_vector,movie2_vector)
    manhattan = distance.cityblock(movie1_vector,movie2_vector)
    manhattan = manhattan.item()
    
    return euclidean,manhattan

In [61]:
x = estimateMovieDistance(movie1,movie2)

In [64]:
x

(308185257.84597576, 471127716.8333333)

In [53]:
from math import dist

print(dist(movie1_vector,movie2_vector))

308185257.84597576


In [31]:
def get_year(movie_id):
    url = "https://www.imdb.com/title/"+movie_id+ "/releaseinfo?ref_=tt_ov_rdat"
    
    print(url)
    try:
        response = requests.get(url)
        result = response.content    
        soup = BeautifulSoup(result, 'html.parser')
        year = soup.find("td", attrs = {"class" : "release-date-item__date"}).getText()
        year = int(year.split(" ")[2]) #split the date by whitespaces. Get the third element, which is the year of release.
        
        return year
    except:
        print("Date not available")
        return 0

In [40]:
def get_runtime(movie_id):
    url = "https://www.imdb.com/title/"+movie_id+ "/technical?ref_=tt_spec_sm"
    
    print(url)
    try:
        response = requests.get(url)
        result = response.content    
        soup = BeautifulSoup(result, 'html.parser')
        
        table = soup.find("table", attrs = {"class" : "dataTable labelValueTable"}).find("tbody")
        
        children = table.contents
        runtime = children[1].contents[3].getText().split("(")[1]
        runtime = runtime.split(" ")[0]
        
        return runtime
    
    except:
        print("Runtime not available")
        return 0

In [16]:
def get_votes(movie_id):
    url = "https://www.imdb.com/title/"+movie_id+ "/ratings/"
    
    print(url)
    try:
        response = requests.get(url)
        result = response.content    
        soup = BeautifulSoup(result, 'html.parser')
        
        votes = soup.find("td", attrs = {"class" : "ratingTable Selected"}).find("div", attrs = {"class":"smallcell"}).getText()
        votes = re.sub("[^0-9]", "", votes)
        return int(votes)
    
    except:
        print("Votes not available")
        return 0

In [41]:
def get_metascore(movie_id):
    url = "https://www.imdb.com/title/"+movie_id+ "/criticreviews/"
    
    print(url)
    try:
        response = requests.get(url)
        result = response.content    
        soup = BeautifulSoup(result, 'html.parser')
        
        metascore = soup.find("div", attrs = {"class" : "metascore_block"}).find("div", attrs = {"class":"metascore_wrap"})
        metascore = metascore.find("span").getText()
        
        return int(metascore)
    
    except:
        print("Metascore not available")
        return 0

In [72]:
def get_num_reviews(movie_id):
    url = "https://www.imdb.com/title/"+movie_id+ "/reviews/"
    
    print(url)
    try:
        response = requests.get(url)
        result = response.content    
        soup = BeautifulSoup(result, 'html.parser')
        
        num_reviews = soup.find("div", attrs = {"class" : "lister"}).find("div", attrs = {"class":"header"})
        num_reviews = num_reviews.find("span").getText()
        num_reviews = re.sub("[^0-9]", "", num_reviews)
        return int(num_reviews)
    
    except:
        print("Number of reviews not available")
        return 0

In [76]:
def get_num_exernal_reviews(movie_id):
    url = "https://www.imdb.com/title/"+movie_id+ "/externalreviews/"
    
    print(url)
    try:
        response = requests.get(url)
        result = response.content    
        soup = BeautifulSoup(result, 'html.parser')
        
        num_reviews_external = soup.find("div", attrs = {"id" : "external_reviews_content"}).find("div", attrs = {"class":"nav"}).getText()
        num_reviews_external = re.sub("[^0-9]", "", num_reviews_external)
        return int(num_reviews_external)
    
    except:
        print("Number of external reviews not available")
        return 0

In [75]:
from selenium import webdriver
def search_movies():

    # This function asks the user for the input of the name of the movie. Searches the name of the movie in imdb search service.
    # Selenium webdriver is used to access to the imdb search service.
    # The function takes as argument the path to search for the executable webdriver. Path is given from the main module where this function is called. 
    # As a result the function returns the direct link of the movie the user wanted to search and the movie id in the imdb database. 
    
    search= input('Search movies:') #name of the movie from user
    website = 'https://www.imdb.com/'
    
    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-logging']) #options to avoid abnormal logging
    
    driver = webdriver.Chrome("driver\chromedriver",options=options)
    
    driver.get(website)
    driver.find_element("id",'suggestion-search').send_keys(search)
    driver.find_element("id", 'suggestion-search-button').click()
    

    table=driver.find_element("xpath",'//*[@id="__next"]/main/div[2]/div[2]/section/div/div[1]/section[2]/div[2]/ul')
    results_search = table.find_elements("class name","ipc-metadata-list-summary-item__tc")
    results = []
    for result in results_search:
        results.append(result.find_element("tag name", "a"))
        #print(result.find_element("tag name", "a"))
    return results

In [81]:
x = search_movies()

Search movies: matriz


<ipython-input-75-9f9f433174f1>:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("driver\chromedriver",options=options)


In [98]:
def get_rating(movie_id):
    url = "https://www.imdb.com/title/"+movie_id
    
    print(url)
    
    try:

        response = requests.get(url)
        result = response.content    
        soup = BeautifulSoup(result, 'html.parser')

        rating = soup.find('div',  attrs = {"data-testid":'hero-rating-bar__aggregate-rating__score'}).find('span', class_ = 'sc-7ab21ed2-1 jGRxWM').getText()
        rating = float(rating)    
        return rating
    except:
        print("Rating is not available")
        return 0

In [99]:
x = get_rating("tt2543164")

https://www.imdb.com/title/tt2543164


## py mongo get data

In [66]:
import pymongo

In [71]:
client = pymongo.MongoClient("localhost",27017)
db = client["movie_recommendations"]
collection = db["movie_recommendation"]

In [72]:
def get_imdb_rec_mongo(movie):
    client = pymongo.MongoClient("localhost",27017)
    db = client["movie_recommendations"]
    collection = db["imdb_rec_temp"]
    doc = collection.find({"movie_id": movie["imdb_id"]})
    data = doc[0]
    df = pd.DataFrame(data["info_recommended"])
    return df

In [74]:
doc = collection.find({"movie_id": "tt0133093"})
data = doc[0]

In [76]:
import pandas as pd

In [79]:
df = pd.DataFrame(data["imdb_recommended"])
df.head(3)

,name,imdb_id,year_released,runtime,imdb_reviews,external_reviews,imdb_rating,metacritic_punctuation,budget,earning_worldwide,earning_US&CA,genres,directors,cast,common_genres_rate,common_directors_rate,common_cast_rate,euclidean_distance,manhattan_distance
0,Inception,tt1375666,2010,148,4648,483,8.8,74,160000000,836848102,292587330,"[Action, Adventure, Sci-Fi]",[Christopher Nolan],"[Leonardo DiCaprio, Joseph Gordon-Levitt, Elli...",1.0,0.0,0.000000,4.006927e+08,5.871363e+08
1,Forrest Gump,tt0109830,1994,142,3015,176,8.8,82,55000000,678226465,330455270,"[Drama, Romance]",[Robert Zemeckis],"[Tom Hanks, Robin Wright, Gary Sinise, Sally F...",0.0,0.0,0.000000,2.639513e+08,3.773840e+08
2,Matrix Reloaded,tt0234215,2003,138,2919,184,7.2,62,150000000,741847937,281576461,"[Action, Sci-Fi]","[Lana Wachowski, Lilly Wachowski]","[Keanu Reeves, Laurence Fishburne, Carrie-Anne...",1.0,1.0,0.222222,3.081853e+08,4.711268e+08


In [80]:
df2 = pd.DataFrame(data["new_recommended"])
df2.head(3)

,name,imdb_id,year_released,runtime,imdb_reviews,external_reviews,imdb_rating,metacritic_punctuation,budget,earning_worldwide,earning_US&CA,genres,directors,cast,common_genres_rate,common_directors_rate,common_cast_rate,euclidean_distance,manhattan_distance
0,Inception,tt1375666,2010,148,4648,483,8.8,74,160000000,836848102,292587330,"[Action, Adventure, Sci-Fi]",[Christopher Nolan],"[Leonardo DiCaprio, Joseph Gordon-Levitt, Elli...",1.0,0.0,0.0,4.006927e+08,5.871363e+08
1,Matrix,tt0133093,1999,136,4874,255,8.7,73,63000000,467222728,172076928,"[Action, Sci-Fi]","[Lana Wachowski, Lilly Wachowski]","[Keanu Reeves, Laurence Fishburne, Carrie-Anne...",1.0,1.0,1.0,0.000000e+00,0.000000e+00
2,Avengers,tt0848228,2012,143,2094,759,8.0,69,220000000,1518815515,623357910,"[Action, Adventure, Sci-Fi]",[Joss Whedon],"[Robert Downey Jr., Chris Evans, Scarlett Joha...",1.0,0.0,0.0,1.155055e+09,1.659877e+09
